In [1]:
%pylab notebook
import datacube
from datetime import datetime, timedelta
from datacube.storage.storage import write_dataset_to_netcdf
from fc.fractional_cover import fractional_cover
import pandas as pd

wofs_db = datacube.compat.read_config(u"""
[datacube]
# Blank implies localhost
db_hostname: 130.56.244.105
db_database: wofscube
db_port: 6432
db_username: dg6911
db_password: _PsmygJVtNJTCAmfg1GmSYE_fNnjrNIT
# If a connection is unused for this length of time, expect it to be invalidated
db_connection_timeout: 60
""")

wofs_config = datacube.config.LocalConfig(wofs_db)
dc = datacube.Datacube(app='fc-tester-example')
wofsdc = datacube.Datacube(app='fc-water-masking', config=wofs_config)

Populating the interactive namespace from numpy and matplotlib


In [2]:
scenes = [
    ('ls8','089','079','2014-09-05'),
]

In [3]:
regression_coefficients = {
    'blue': [0.00041,0.97470],
    'green': [0.00289,0.99779],
    'red': [0.00274,1.00446],
    'nir': [0.00004,0.98906],
    'swir1': [0.00256,0.99467],
    'swir2': [-0.00327,1.02551]
}

In [4]:
measurements = [
    {'name': 'BS',
     'src_var': 'BS',
     'units': "percent",
     'dtype': 'int8',
     'nodata': -1,
     'zlib': True,
     'fletcher32': True,
     'attrs':{
         'long_name': "Bare ground fraction percentage",
         'comment': "Fraction of bare ground including rock, bare and disturbed soil, within the Landsat pixel as percentages",
         'alias': "bare",
         'coverage_content_type': 'modelResult'
     },
      'aliases':["bare"]},
    {'name': 'PV',
      'src_var': 'PV',
      'units': "percent",
      'dtype': 'int8',
      'nodata': -1,
      'zlib': True,
      'fletcher32': True,
      'attrs':{
          'long_name': "Green cover fraction percentage",
          'comment': "Fraction of green cover including green groundcover and green leaf material over all strata, within the Landsat pixel, as percentages",
          'alias': "green",
          'coverage_content_type': 'modelResult'
          },
      'aliases': ["green_veg"]},
    {'name': 'NPV',
      'src_var': 'NPV',
      'units': "percent",
      'dtype': 'int8',
      'nodata': -1,
      'zlib': True,
      'fletcher32': True,
      'attrs':{
          'long_name': "Non-green cover fraction percentage",
          'comment': "Fraction of non green cover including litter, dead leaf and branches over all strata, within the Landsat pixel, as percentages",
          'alias': "dead",
          'coverage_content_type': 'modelResult'
         },
      'aliases': ["dead_veg"]},
    {'name': 'UE',
      'src_var': 'UE',
      'units': "1",
      'dtype': 'int8',
      'nodata': -1,
      'zlib': True,
      'fletcher32': True,
      'attrs':{
          'long_name': "Unmixing error",
          'comment': "Eclidean Norm of the Residual Vector",
          'alias': "err",
          'coverage_content_type': 'qualityInformation'
          },
      'aliases': ["err"]}
]

In [16]:
def write_old_and_new_fc(sensor, path, row, date):
    target_day = datetime.strptime(date, '%Y-%m-%d')
    day_before = (target_day - timedelta(days=1)).strftime('%Y-%m-%d')
    day_after = (target_day + timedelta(days=1)).strftime('%Y-%m-%d')    
    target_dir = '/g/data1/r78/dg6911/jrsrp-fc/'+path+'_'+row+'/'
    print('path: ' + path + ' row:' + row + ' date:' + date)    
    print('target dir: '+target_dir)
    date_nodash = date.replace('-','')
    target_file = '_'+sensor+'_'+path+'_'+row+'_'+date_nodash+'.nc'
    #old_target = target_dir + 'old_fc' + target_file
    #nbart_target = target_dir + 'nbart' + target_file
    new_target = target_dir + 'new_end2_fc' + target_file
    
    source_filter = dict(product=sensor+'_nbart_scene', sat_path=path, sat_row=row)
    #source_filter = dict(sat_path=path, sat_row=row)
#     try:
#         print('reading old FC')
#         data = dc.load(product=sensor+'_fc_albers', time=(day_before, day_after), source_filter=source_filter)
    
#         print('writing old FC')
#         write_dataset_to_netcdf(data, old_target)
#     except:
#         print('could not find: '+sensor+'_p'+path+'_r'+row+'_'+date)
#         print('error:', sys.exc_info()[0])
    
    print('reading NBART')
    data = dc.load(product=sensor+'_nbart_albers', time=(day_before, day_after), source_filter=source_filter)
    
    print('reading WOFS')
    return wofsdc.load(product='old_wofs', time=(day_before, day_after), like=data)
    
    #write_dataset_to_netcdf(data, nbart_target)
#     if(sensor == 'ls8'):
#          use_coefficients=regression_coefficients
#     else:
#          use_coefficients=None
#     print('calculating new FC')
#     data = data.squeeze()
#     new_fc_data = fractional_cover(nbar_tile=data, measurements=measurements, regression_coefficients=use_coefficients)
    
#     print('writing new FC')
#     write_dataset_to_netcdf(new_fc_data, new_target)
    

In [17]:
for scene in scenes:
    wofs = write_old_and_new_fc(*scene)

path: 089 row:079 date:2014-09-05
target dir: /g/data1/r78/dg6911/jrsrp-fc/089_079/
reading NBART
reading WOFS


In [22]:
write_dataset_to_netcdf(wofs, '/g/data1/r78/dg6911/jrsrp-fc/wofs.nc')

In [24]:
wofs['water'] = wofs.water.where(wofs.water==128)

In [26]:
write_dataset_to_netcdf(wofs, '/g/data1/r78/dg6911/jrsrp-fc/wofs2.nc')